In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
from tensorflow import keras

from keras.utils import np_utils
import keras.backend as K

In [41]:
#df = pd.read_csv('../data/singe_cell.csv',  index_col=0)
df = pd.read_csv('../data/original.csv',  index_col=0)

In [42]:
y = np.array(df['group'])

In [43]:
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
dummy_y = np_utils.to_categorical(encoded_Y)

In [64]:
X = df.drop(['group'], axis = 1)
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [65]:
x_train , x_test , y_train , y_test =train_test_split(X,
                            dummy_y,test_size =0.2, random_state=105)

In [66]:
import keras.backend as K

def relu_nan(x):
    return K.relu(x, max_value=None) * K.cast(K.not_equal(x, K.constant(float('nan'))), K.floatx())


In [ ]:
latent_space = 2

input_layer = keras.Input(shape = (199,))
encoded = layers.Dense(128, activation='relu')(input_layer)
encoded = layers.Dense(64, activation='relu')(encoded)
encoded = layers.Dense(latent_space, activation='relu')(encoded)

decoded = layers.Dense(64, activation='relu')(encoded)
decoded = layers.Dense(128, activation='relu')(encoded)
decoded = layers.Dense(199, activation='linear')(decoded)

In [ ]:
encoder = keras.Model(input_layer, encoded)
autoencoder = keras.Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

autoencoder.fit(X, X,
                epochs=100,
                batch_size=32,
                validation_split = 0.1)

In [67]:
model = Sequential()
#model.add(tf.keras.layers.Masking(mask_value=0., input_dim = 199))
model.add(Dense(units=128, activation= relu_nan, input_dim = 199)) # add layers to neural network
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=4, activation='softmax'))
model.compile(loss="categorical_crossentropy",
              optimizer= "adam")

In [68]:
model.fit(x=x_train,
        y=y_train,
        epochs=50,
        batch_size = 1
        )

Epoch 1/50
80/80 [==============================] - 0s 714us/step - loss: nan
Epoch 2/50
80/80 [==============================] - 0s 616us/step - loss: nan
Epoch 3/50
80/80 [==============================] - 0s 601us/step - loss: nan
Epoch 4/50
80/80 [==============================] - 0s 611us/step - loss: nan
Epoch 5/50
80/80 [==============================] - 0s 608us/step - loss: nan
Epoch 6/50
80/80 [==============================] - 0s 609us/step - loss: nan
Epoch 7/50
80/80 [==============================] - 0s 617us/step - loss: nan
Epoch 8/50
80/80 [==============================] - 0s 603us/step - loss: nan
Epoch 9/50
80/80 [==============================] - 0s 607us/step - loss: nan
Epoch 10/50
80/80 [==============================] - 0s 620us/step - loss: nan
Epoch 11/50
80/80 [==============================] - 0s 649us/step - loss: nan
Epoch 12/50
80/80 [==============================] - 0s 648us/step - loss: nan
Epoch 13/50
80/80 [==============================] - 0s 633us

In [35]:
def na_custom_loss(y_true, y_pred):
    y_true_mask = tf.where(tf.math.is_nan(y_true), tf.zeros_like(y_true), y_true)
    return K.categorical_crossentropy(y_pred, y_true_mask)

In [66]:
y_true = tf.constant([
    [0.1, 1, np.nan, 0.5],
    [0.4, 0.2, 0.8, 0],
    [np.nan, 1.3, np.nan, 0.2],
    [0.1, 0.1, 0.6, np.nan],
])

y_pred = tf.constant([
    [0.7, 0.7, 0.1, 0.3],
    [0.6, 0.1, 0.1, 0],
    [0.4, 0.9, 0.9, 0.2],
    [0.1, 0.4, 0.4, 1.2],
])

#loss = na_custom_loss(y_true, y_pred)


In [67]:
tf.where(tf.math.is_nan(y_true), tf.zeros_like(y_true), y_true)

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[0.1, 1. , 0. , 0.5],
       [0.4, 0.2, 0.8, 0. ],
       [0. , 1.3, 0. , 0.2],
       [0.1, 0.1, 0.6, 0. ]], dtype=float32)>

In [ ]:
X =  np.nan_to_num(X, nan = 9999)

In [43]:
na_custom_loss(y_pred, y_pred) # loook more into it. 

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([2.1488113 , 0.58849764, 2.979178  , 2.3025734 ], dtype=float32)>

In [37]:
model.compile(loss = na_custom_loss,
              optimizer= "adam")

model.fit(x=x_train,
        y=y_train,
        epochs=50,
        validation_split = 0.2
        )

NameError: name 'na_custom_loss' is not defined

In [38]:
import numpy as np
# Define the model architecture
model = keras.models.Sequential([
    # Add a masking layer to ignore NaN values
    keras.layers.Masking(mask_value=-9999, input_shape=(199,)),
    # Add a dense layer with ReLU activation
    keras.layers.Dense(128, activation='relu'),
    # Add a dropout layer to reduce overfitting
    keras.layers.Dropout(0.2),
    # Add another dense layer with ReLU activation
    keras.layers.Dense(64, activation='relu'),
    # Add a final dense layer with softmax activation for multi-class classification
    keras.layers.Dense(4, activation='softmax')
])

# Compile the model with categorical cross-entropy loss and Adam optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary
model.summary()


model.fit(x=x_train,
        y=y_train,
        epochs=50,
        validation_split = 0.2
        )

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_3 (Masking)         (None, 199)               0         
                                                                 
 dense_15 (Dense)            (None, 128)               25600     
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_16 (Dense)            (None, 64)                8256      
                                                                 
 dense_17 (Dense)            (None, 4)                 260       
                                                                 
Total params: 34,116
Trainable params: 34,116
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
2/2 [==============================] - 

2/2 [==============================] - 0s 16ms/step - loss: 53.1108 - accuracy: 0.9531 - val_loss: 2831.4834 - val_accuracy: 0.2500


In [39]:
model.predict(x_test)

1/1 [==============================] - 0s 45ms/step


array([[0.0000000e+00, 0.0000000e+00, 1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00, 2.8969705e-11, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+

In [34]:
y_test

array([[0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.]], dtype=float32)